In [ ]:
#!pip install autogluon.tabular autogluon.vision

In [1]:
# 

In [3]:
# Оптимизация TensorFlow
import tensorflow as tf
tf.config.optimizer.set_jit(True)  # Включение XLA-компиляции
tf.config.threading.set_intra_op_parallelism_threads(8)
#tf.config.threading.set_inter_op_parallelism_threads(8)

# Для GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            #tf.config.experimental.set_memory_limit(gpu, 8192)  # Лимит 8GB
    except RuntimeError as e:
        print(e)

In [5]:
# Текущая директория + путь к датасету
import os
HOME = os.getcwd()
print(HOME)
print('-----------------------')

# Пути
IMAGE_PATH = "/home/oleg/projects/datasets/construction"
os.listdir(IMAGE_PATH)

# ============================================================================
# УСТАНОВКА AUTOGLUON (если не установлен)
# ============================================================================
"""
!pip install autogluon.core
!pip install autogluon.vision
!pip install timm
"""

# ============================================================================
# ИМПОРТЫ
# ============================================================================
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset
from autogluon.core.utils.loaders import load_zip
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import time
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# ПРОВЕРКА ДАННЫХ
# ============================================================================
print("📁 Проверка структуры данных:")
classes = os.listdir(IMAGE_PATH)
print(f"🎯 Найдено классов: {len(classes)}")
print(f"🏷️  Классы: {classes}")

# Подсчет количества файлов в каждом классе
print("\n📊 Статистика по классам:")
for class_name in classes:
    class_path = os.path.join(IMAGE_PATH, class_name)
    if os.path.isdir(class_path):
        num_files = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"   {class_name}: {num_files} изображений")

# ============================================================================
# НАСТРОЙКА АВТОГЛУОНА
# ============================================================================
# Создаем ImageDataset из папки
dataset = ImageDataset.from_folder(IMAGE_PATH)

print(f"\n📦 Размер датасета: {len(dataset)} изображений")
print(f"🎯 Количество классов: {dataset.num_classes}")

# ============================================================================
# РАЗДЕЛЕНИЕ НА TRAIN/TEST
# ============================================================================
train_data, test_data, _ = dataset.random_split(test_size=0.2, random_state=42)

print(f"\n📊 Разделение данных:")
print(f"   Обучающая выборка: {len(train_data)} изображений")
print(f"   Тестовая выборка: {len(test_data)} изображений")

# ============================================================================
# СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ
# ============================================================================
print("\n🚀 Запуск AutoGluon...")

# Настройки обучения
time_limit = 3600  # 1 час на поиск
presets = 'medium_quality_faster_train'  # Быстрое обучение с хорошим качеством

# Создаем предсказатель
predictor = ImagePredictor(
    verbosity=2,  # Подробный вывод
    log_to_file=True  # Логи в файл
)

# Засекаем время
start_time = time.time()

# Обучаем модель
predictor.fit(
    train_data=train_data,
    time_limit=time_limit,
    presets=presets,
    hyperparameters={
        'model': ag.Categorical('resnet50', 'efficientnet_b0'),
        'lr': ag.Real(1e-4, 1e-2, log=True),
        'batch_size': ag.Categorical(16, 32)
    }
)

training_time = time.time() - start_time
print(f"⏰ Время обучения: {training_time/60:.2f} минут")

# ============================================================================
# ОЦЕНКА МОДЕЛИ
# ============================================================================
print("\n📊 Оценка модели на тестовых данных...")

# Предсказания на тестовых данных
test_predictions = predictor.predict(test_data)
test_probabilities = predictor.predict_proba(test_data)

# Оценка качества
test_metrics = predictor.evaluate(test_data)
print(f"✅ Точность на тесте: {test_metrics['accuracy']:.4f}")
print(f"📊 Полная статистика: {test_metrics}")

# ============================================================================
# ДЕТАЛЬНАЯ СТАТИСТИКА
# ============================================================================
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# True labels
true_labels = [test_data[i][1] for i in range(len(test_data))]

print("\n🎯 Classification Report:")
print(classification_report(true_labels, test_predictions, target_names=classes))

# Матрица ошибок
plt.figure(figsize=(10, 8))
cm = confusion_matrix(true_labels, test_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# ============================================================================
# ПРЕДСКАЗАНИЯ НА НОВЫХ ДАННЫХ (пример)
# ============================================================================
print("\n🔮 Примеры предсказаний:")

# Берем несколько примеров из тестовой выборки
sample_indices = np.random.choice(len(test_data), min(5, len(test_data)), replace=False)

for idx in sample_indices:
    image, true_label = test_data[idx]
    predicted_label = test_predictions[idx]
    probability = max(test_probabilities[idx])
    
    print(f"   🖼️  Истинный класс: {true_label}")
    print(f"   🤖 Предсказанный: {predicted_label}")
    print(f"   📈 Вероятность: {probability:.4f}")
    print(f"   {'✅ Правильно' if true_label == predicted_label else '❌ Ошибка'}")
    print("   " + "-" * 40)

# ============================================================================
# СОХРАНЕНИЕ МОДЕЛИ
# ============================================================================
model_path = "./autogluon_construction_model"
predictor.save(model_path)
print(f"\n💾 Модель сохранена в: {model_path}")

# ============================================================================
# ЗАГРУЗКА МОДЕЛИ (пример)
# ============================================================================
# loaded_predictor = ImagePredictor.load(model_path)

# ============================================================================
# ИНФОРМАЦИЯ О ЛУЧШЕЙ МОДЕЛИ
# ============================================================================
print("\n🏆 Информация о лучшей модели:")
leaderboard = predictor.leaderboard(test_data)
print(leaderboard)

# Лучшая модель
best_model = predictor.get_model_best()
print(f"🤖 Лучшая модель: {best_model}")

# ============================================================================
# ФИНАЛЬНАЯ СТАТИСТИКА
# ============================================================================
print("\n🎉 ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:")
print(f"   🎯 Точность: {test_metrics['accuracy']:.4f}")
print(f"   ⏰ Время обучения: {training_time/60:.2f} мин")
print(f"   📦 Размер тренировочных данных: {len(train_data)}")
print(f"   🧪 Размер тестовых данных: {len(test_data)}")
print(f"   🏷️  Количество классов: {len(classes)}")
print(f"   💾 Модель сохранена: {model_path}")

print("\n✨ AutoGluon завершил работу!")

/home/oleg/projects/ML
-----------------------


ImportError: cannot import name '_approximate_mode' from 'sklearn.utils' (/home/oleg/miniconda3/envs/TF/lib/python3.10/site-packages/sklearn/utils/__init__.py)